In [ ]:
# author f.corradi@tue.nl z.shen1@student.tue.nl
# Training Spiketransformer solving image classification
import os
from datetime import date

folder_src = "./result/"
srbm_fpga = ''
datasets = ["MNIST", "genomic", "DVS128", "PSMNIST", "NMNIST", "cifar10"]
index_dataset = 2
dataset = datasets[index_dataset]

today = date.today()
folder = folder_src + f"{dataset}/"
try:
    os.stat(folder)
except:
    os.mkdir(folder)

from model import *
# from model import sdt

cuda:0
cuda:0
Device in use: cuda:0


/home/federico/aienv/aienv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/federico/aienv/aienv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/federico/aienv/aienv/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
import ipdb

In [3]:
import numpy as np
import gc
from contextlib import suppress
from collections import OrderedDict
import torch
from pytorch_lamb import Lamb
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
from torch import Generator
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR, MultiStepLR, CosineAnnealingLR
# from torch.optim.lr_scheduler import ReduceLROnPlateau, LambdaLR
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from torch.utils.data import TensorDataset, DataLoader, Dataset, Subset, ConcatDataset, random_split
from timm.data import create_loader
from timm.loss import BinaryCrossEntropy
from timm.scheduler import create_scheduler
from timm.scheduler.cosine_lr import CosineLRScheduler
from timm.scheduler.multistep_lr import MultiStepLRScheduler
from timm.scheduler.plateau_lr import PlateauLRScheduler
from timm.scheduler.poly_lr import PolyLRScheduler
from timm.scheduler.step_lr import StepLRScheduler
from timm.scheduler.tanh_lr import TanhLRScheduler
from torch.optim.lr_scheduler import LambdaLR
from tqdm import tqdm
import torchinfo
import dvs_utils
from memory_profiler import profile
from module.lif_neuron import lif_neuron, LIFNeuron

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from pylab import *


In [4]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

/tmp/ipykernel_2460109/2736003858.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [5]:
from spikingjelly.clock_driven import functional
from torchvision.datasets.mnist import MNIST
from spikingjelly.datasets.cifar10_dvs import CIFAR10DVS
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture
from spikingjelly.datasets.n_mnist import NMNIST

In [6]:
# configurations
genomic_folders = ['D1-mild-bottleneck/', 'D2-severe-bottleneck/', 'D3-recent-migration/', 'D4-old-migration/', 'D5-low-intensity-recombination/', 'D6-high-intensity-recombination/', 'D7-Recombination-hotspot-intensity2/', 'D8-Recombination-hotspot-intensity10/']
data_index = 0
batch_size = 20
num_workers = 0
num_heads = 1
mlp_ratio = 2
layers = 2
in_channels = 1
spike_mode = "if"
input_extend = 1
dvs_mode = False
model_type="tsdt"
spsmodule="bsps"
encodermodule="encoder"
train_threshold = False
if dataset == "MNIST":
    in_channels = 3

In [7]:
dataset_size = {"MNIST" : 28, "DVS128" : 128, "genomic" : 128, "PSMNIST" : 98, "cifar10":64, "NMNIST":34, "cifar10":128}
dataset_classes = {"MNIST" : 10, "DVS128" : 11, "genomic" : 2, "PSMNIST" : 10, "cifar10":10, "NMNIST":10, "cifar10":10}

image_size = dataset_size[dataset]
num_classes = dataset_classes[dataset]
num_epoch = 50
drop_rate = 0

if dataset == "DVS128":
    time_steps = 16
    embed_dims = 256
    patch_size = 8
    input_extend = 1
    num_epoch = 200
    thresholds = [128/128, 128/128, 128/128]
    pooling_stat = "1111"
elif dataset == "NMNIST":
    time_steps = 4
    embed_dims = 64
    patch_size = 8
    batch_size = 500
    input_extend = 1
    num_epoch = 100
    thresholds = [128/128, 128/128, 128/128]
    pooling_stat = "0011"
elif dataset == "PSMNIST":
    time_steps = 4
    embed_dims = 64
    batch_size = 500
    thresholds = [128/128, 128/128, 128/128]
    patch_size = 2
    pooling_stat = "0011"
elif dataset == "genomic":
    num_epoch = 200
    batch_size = 200
    time_steps = 2
    embed_dims = 64
    patch_size = 16
    thresholds = [128/128, 128/128, 128/128]
    pooling_stat = "1111"
elif dataset == "cifar10":
    time_steps = 16
    embed_dims = 256
    patch_size = 4
    batch_size = 20
    num_epoch = 150
    pooling_stat = "1111"
    thresholds = [128/128, 128/128, 128/128]
else:
    time_steps = 8
    embed_dims = 64
    patch_size = 4
    batch_size = 500
    thresholds = [128/128, 128/128, 128/128]
    pooling_stat = "0011"

In [8]:
import spikingjelly.datasets as sjds
events_per_frame = 15000
events_per_frame1 = 10000
def integrate_fixed_15000_events(events, H, W, events_per_frame=events_per_frame):
    t, x, y, p = (events[key] for key in ('t', 'x', 'y', 'p'))
    total_events = len(t)
    num_frames = int(np.ceil(total_events / events_per_frame))
    frames = np.zeros([num_frames, 2, H, W], dtype=np.float32)

    for i in range(num_frames):
        start_index = i * events_per_frame
        end_index = min((i + 1) * events_per_frame, total_events)
        for j in range(start_index, end_index):
            if p[j] == 1:
                frames[i, 1, y[j], x[j]] += 1
            else:
                frames[i, 0, y[j], x[j]] += 1

    return frames

def integrate_fixed_10000_events(events, H, W, events_per_frame=events_per_frame):
    t, x, y, p = (events[key] for key in ('t', 'x', 'y', 'p'))
    total_events = len(t)
    num_frames = int(np.ceil(total_events / events_per_frame1))
    frames = np.zeros([num_frames, 2, H, W], dtype=np.float32)

    for i in range(num_frames):
        start_index = i * events_per_frame1
        end_index = min((i + 1) * events_per_frame1, total_events)
        for j in range(start_index, end_index):
            if p[j] == 1:
                frames[i, 1, y[j], x[j]] += 1
            else:
                frames[i, 0, y[j], x[j]] += 1

    return frames

In [9]:
dataset

'DVS128'

In [10]:
if dataset == "DVS128":
#     dataset_train = DVS128Gesture(
#         "data/DVS128/",
#         train=True,
#         data_type="frame",
#         frames_number=time_steps,
#         split_by="number",
#     )
#     dataset_test = DVS128Gesture(
#         "data/DVS128/",
#         train=False,
#         data_type="frame",
#         frames_number=time_steps,
#         split_by="number",
#     )
    dataset_train = DVS128Gesture(
        "data/DVS128/",
        train=True,
        data_type="frame",
        custom_integrate_function=integrate_fixed_15000_events
    )
    dataset_test = DVS128Gesture(
        "data/DVS128/",
        train=False,
        data_type="frame", 
        custom_integrate_function=integrate_fixed_15000_events
    )
elif dataset == "MNIST":
    transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
    ])
    dataset_train = MNIST(
        "data/mnist/",
        train = True,
        transform=transform
    )
    dataset_test = MNIST(
        "data/mnist/",
        train = False,
        transform=transform
    )
elif dataset == "cifar10":
    dataset_tmp = CIFAR10DVS(
        "../snntransformer/data/cifar10-dvs/",
        data_type="frame",
        custom_integrate_function=integrate_fixed_15000_events
    )
    dataset_train, dataset_test = dvs_utils.split_to_train_test_set(
        0.8, dataset_tmp, 42
    )
elif dataset == "genomic":
    dataset_train = "../biosnn-master/data/Row&column/" + genomic_folders[data_index] + "train/images/"
    dataset_test = "../biosnn-master/data/Row&column/" + genomic_folders[data_index] + "test/images/"
else:
    class ReshapeTransform:
        def __init__(self, new_size):
            self.new_size = new_size

        def __call__(self, img):
            return img.view(self.new_size)
        
    class PixelPermuteTransform:
        def __init__(self, perm):
            self.perm = perm

        def __call__(self, img):
            img = img[:,self.perm, :]
            return img
    np.random.seed(42)
    rng = np.random.default_rng()
    perm = rng.permutation(image_size)
    print(perm)
    transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,)),
    ReshapeTransform((-1, image_size, 28*28//image_size)),
    PixelPermuteTransform(perm),
    ])
    dataset_train = MNIST(
        "data/mnist/",
        train=True,
        transform=transform
    )
    dataset_test = MNIST(
        "data/mnist/",
        train=False,
        transform=transform
    )

loader_train, loader_eval, train_idx = None, None, None
    

The directory [data/DVS128/integrate_fixed_15000_events] already exists.
The directory [data/DVS128/integrate_fixed_15000_events] already exists.


In [11]:
if dataset != "cifar10":
    combined_dataset = ConcatDataset([dataset_train, dataset_test])
    train_size = int(0.8 * len(combined_dataset))
    test_size = len(combined_dataset) - train_size
    generator = torch.Generator()
    seed = np.random.randint(low=0, high=100)
    print(seed)
    generator.manual_seed(43)
    dataset_train, dataset_test = random_split(combined_dataset, [train_size, test_size], generator=generator)

38


In [12]:
if dataset == "DVS128" or dataset == "SHD" or dataset == "cifar10" or dataset == "NMNIST":
#     loader_train = DataLoader(
#         dataset_train,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=num_workers,
#         pin_memory=False,
#     )
#     loader_test = DataLoader(
#         dataset_test,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=num_workers,
#         pin_memory=False,
#     )
#     def custom_collate_fn(batch):
#         batch = [(item[0][:4, :, :, :], item[1]) for item in batch]
#         return torch.utils.data.dataloader.default_collate(batch)
    def custom_collate_fn(batch):
        max_timesteps = time_steps
        padded_batch = []

        for item in batch:
            data, label = item
            if isinstance(data, np.ndarray):
                data = torch.from_numpy(data).float()

            current_timesteps = data.size(0)

            if current_timesteps < max_timesteps:
                padding_size = (0, 0, 0, 0, 0, 0, 0, max_timesteps - current_timesteps)
                padded_data = torch.nn.functional.pad(data, pad=padding_size, mode='constant', value=0)
            else:
                padded_data = data[:max_timesteps]

            padded_batch.append((padded_data, label))

        return torch.utils.data.dataloader.default_collate(padded_batch)
    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    loader_test = DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    print(len(loader_train))
elif dataset == "genomic": 
    transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                   transforms.RandomCrop((128, 128,), padding=None, 
                                                         pad_if_needed=False, 
                                                         fill=0, 
                                                         padding_mode='constant'),
                                   transforms.ToTensor()])
    train_data = torchvision.datasets.ImageFolder(root=dataset_train, transform=transform)
    test_data = torchvision.datasets.ImageFolder(root=dataset_test, transform=transform)
    loader_train = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=False,
    )
    loader_test = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=False,
    )
else:
    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=False,
    )
    loader_test = DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False,
    )
print("Create dataloader: {}".format(dataset)) 


59
Create dataloader: DVS128


In [13]:
print(dataset)

DVS128


In [14]:
examples = enumerate(loader_train)
batch_idx, (example_data, example_targets) = next(examples)
# print(example_data.shape)
if dataset == "DVS128" or dataset == "cifar10":
    example_data = example_data.transpose(0, 1)
elif dataset == "SHD":
    time_steps=len(example_data[0])
print(torch.max(example_data))
example_data.shape

tensor(102.)


torch.Size([16, 20, 2, 128, 128])

In [15]:
in_channels = 2 if dataset == "DVS128" or dataset == "NMNIST" or dataset == "cifar10" else 1

In [16]:
if dataset == "PSMNIST":
    image_size_h = image_size 
    image_size_w = 28*28//image_size
else:
    example_data = example_data.cpu().detach().numpy()
    image_size_h = image_size
    image_size_w = image_size

example_data.shape

(16, 20, 2, 128, 128)

In [17]:
if dataset != "DVS128" and dataset != "NMNIST" and dataset != "cifar10":
    fig, idx = plt.subplots(2, 2)
    subfigs = []
    for i in range(2):
        for j in range(2):
            subfigs.append(idx[i, j])

    for i in range(4):
        subfigs[i].imshow(example_data[i][0], cmap='gray', interpolation='none')
        subfigs[i].set_title('{}'.format(type[example_targets[i]]))
        subfigs[i].set_xticks([])
        subfigs[i].set_yticks([])
        plt.tight_layout()

In [18]:
criterion = nn.CrossEntropyLoss()

train_loss_fn = criterion.cuda()
test_loss_fn = criterion.cuda()

In [19]:
if model_type=="bsdt":
    model = bsdt(
        dataset=dataset,
        spsmodule=spsmodule,
        encodermodule=encodermodule,
        T=time_steps,
        img_size_h=image_size, 
        img_size_w=image_size_w,
        in_channels=in_channels,
        num_classes=num_classes,
        patch_size=patch_size,
        embed_dims=embed_dims,
        num_heads=num_heads,
        pooling_stat=pooling_stat,
        mlp_ratio=mlp_ratio,
        threshold_head=thresholds[0],
        threshold_sps=thresholds[1],
        threshold_enc=thresholds[2],
        depths=layers,
        spike_mode=spike_mode,
        dvs_mode=dvs_mode,
        train_threshold=train_threshold,
    )
elif model_type=="tsdt":
    model = tsdt(
        dataset=dataset,
        spsmodule=spsmodule,
        encodermodule=encodermodule,
        T=time_steps,
        img_size_h=image_size, 
        img_size_w=image_size_w,
        in_channels=in_channels,
        num_classes=num_classes,
        patch_size=patch_size,
        embed_dims=embed_dims,
        num_heads=num_heads,
        pooling_stat=pooling_stat,
        mlp_ratio=mlp_ratio,
        threshold_head=thresholds[0],
        threshold_sps=thresholds[1],
        threshold_enc=thresholds[2],
        depths=layers,
        train_threshold=train_threshold,
    )
image_size_w

128

In [20]:
in_channels

2

In [21]:
_logger = logging.getLogger("train")
stream_handler = logging.StreamHandler()
format_str = "%(asctime)s %(levelname)s: %(message)s"
stream_handler.setFormatter(logging.Formatter(format_str))
_logger.addHandler(stream_handler)
_logger.propagate = False

In [22]:
base_params = [p for p in model.parameters() if p.requires_grad]

In [23]:
train_loss_list = []
train_eval_loss_list = []
test_loss_list = []
def train(model, loader, test_loader, optimizer, scheduler=None, num_epochs=50, chan=1, precision_epochs=40, load=False, path=None):
    best_acc = 0
#     clipper = bsdt.Clipper()
    
    if load:
        model.load_state_dict(torch.load(path))
        
    acc_state = 0

    for epoch in tqdm(range(num_epochs)):
        train_acc = 0.0
        train_loss_sum = 0
        optimizer.zero_grad()
        predictions = []
        if epoch == 0:
            print(len(loader))
        
        for i, (images, labels) in enumerate(loader):
#             if i >= 80:
#                 break
            labels = labels#.clone().view((-1, labels.shape[0]))
            
            if len(images.shape) == 3:
                images.unsqueeze_(dim=1)
                
            model.train()
            
            with autocast():
                x, y, z = model(images)

                train_loss = criterion(x.to(device), labels.to(device))


            

                pred_ = x.argmax(axis=1)
                predictions.append(pred_.data.cpu().numpy())
            scaler.scale(train_loss).backward()
            
            # gradient clamp
            torch.nn.utils.clip_grad_norm_(
                model.parameters(),
                max_norm=1000,
                norm_type=2,
            )
            
            
            train_loss_sum += train_loss.item()
            predicted = pred_.cpu()
            if i == 0:
                print('\n', 'Starting Epoch:', epoch)
                print(model.lif_y_1.threshold.data)
#                 print(y[0, 0:3, :], '\n')
#                 print(y[0, 0:3, :], '\n')
                print(model.patch_embed.conv1.weight[0][0])
#                 print(z[0, 0:3, 0, :])
#                 print(z[0:3, :].max())
#                 print(' ', labels, '\n', pred_)
                # for module in model.modules():
                #     if isinstance(module, LIFNeuron):
                #         print(module, module.threshold.grad)
#                 print("Threshold grad:", model.patch_embed.lif_infor.threshold.grad)
#                 print("Threshold value:", model.patch_embed.lif_infor.threshold.data)
            temp_acc = (predicted == labels.cpu()).sum()
            train_acc += (predicted == labels.cpu()).sum()
#             if i % 10 == 0:
#                 print(f"Iteration {i}")
#                 print(torch.cuda.memory_summary(device=None, abbreviated=True))

            scaler.step(optimizer)
            scaler.update()
#             for module in model.modules():
#                 if isinstance(module, LIFNeuron):
#                     module.threshold_positive()
            for param_group in optimizer.param_groups:
                param_group['lr'] = max(param_group['lr'], 1e-10)
            torch.cuda.empty_cache()
            
        if (epoch >= precision_epochs):
            low_precision_state = low_precision(model.state_dict(), precision=n_bits)
            high_precision_state = model.state_dict()
            if epoch == precision_epochs:
                torch.save(model.state_dict(), path)
                acc_state = 0
                path = folder +"dataset_" + str(data_index+1) + "_timestep_"+str(time_steps)+"_head_"+str(num_heads)+"_mlp_ratio_"+str(mlp_ratio)+"_encoder_"+str(layers)+"_embed_dims_"+str(embed_dims)+"_nbits_"+str(n_bits)+"_ep_"+str(num_epoch)+"_lr(e-3)_"+str(learning_rate * 1000)+ "_low_precision"+".pth"
                for param_group in optimizer.param_groups:
                    param_group['lr'] = 1e-3
                    # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=3, verbose=True)
#                 scheduler = StepLR(optimizer, step_size=10, gamma=.8)
            model.load_state_dict(low_precision_state)
            model.to(device)

#         train_acc = train_acc.data.cpu().numpy()
        test_acc, train_eval_acc, test_loss = test(model, loader, test_loader)
        if test_acc > acc_state:
            torch.save(model.state_dict(), path)
            acc_state = test_acc
            print('Checkpoint saved')
        train_loss = train_loss_sum/len(loader)
        train_loss_list.append(train_loss)
        # train_eval_loss_list.append(train_eval_loss)
        test_loss_list.append(test_loss)
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1)
#         optimizer.step()
#         scheduler.step(train_loss)
        scheduler.step(epoch)

        for p in model.parameters():
            p.data.clamp_(min=-2, max=2)
        print("Highest test acc: {:.4f}".format(acc_state))
        print("epoch: {:3d}, Train Loss: {:.4f}, Train Acc: {:.4f}, Last batch acc: {:.4f} Learning rate: {}".format(epoch,
                                                                           train_loss,
                                                                           train_acc/(batch_size)/len(loader),
                                                                            temp_acc/batch_size,
                                                                            optimizer.param_groups[0]['lr']))
#         torch.cuda.empty_cache()
def evaluate(model, loader):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        if len(images.shape) == 3:
            images = images.unsqueeze(dim=1)
        outputs, _, _ = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item() * images.size(0)  # 乘以当前批次的大小
        pred_ = outputs.argmax(dim=1)
        if i == 0:
            print(' ', labels, '\n', pred_)
        total_correct += (outputs.argmax(dim=1) == labels).sum().item()
        total_samples += images.size(0)

    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

def test(model, train_loader, test_loader):
    model.eval()
    print("Train eval: ")
    train_loss, train_acc = evaluate(model, train_loader)
    print("Test: ")
    test_loss, test_acc = evaluate(model, test_loader)

    print(f'Train eval Loss: {train_loss:.4f}, Train eval Acc: {train_acc:.4f}')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
    return test_acc, train_acc, test_loss
        
# def test(model, train_loader, test_loader, chan=1):
#     model.eval()
#     with torch.no_grad():
#         train_test_accuracy_n = []
#         test_accuracy_n = []
#         train_test_loss_sum = 0.0
#         test_loss_sum = 0.0
#         examples_b = []
#         for i, (images, labels) in enumerate(train_loader):
#             images = images.to(device)
#             labels = labels.to(device)

#             if len(images.shape) == 3:
#                 images.unsqueeze_(dim=1)
#             pred_, y, z = model(images)
# #             print(pred_.shape)
#             train_test_loss = criterion(pred_.to(device), labels.to(device))
#             pred_ = pred_.argmax(axis=1)
#             train_test_acc = (pred_.to(device) == labels.to(device)).sum().data.cpu().numpy() / float(len(pred_))
#             train_test_loss_sum += train_test_loss.detach().cpu().numpy()
#             train_test_accuracy_n.append(train_test_acc)
        

#         for i, (images, labels) in enumerate(test_loader):
#             images = images.to(device)
#             labels = labels.to(device)

#             if len(images.shape) == 3:
#                 images.unsqueeze_(dim=1)
#             pred_, y, z = model(images)
# #             print(pred_.shape)
#             test_loss = criterion(pred_.to(device), labels.to(device))
#             pred_ = pred_.argmax(axis=1)
#             test_acc = (pred_.to(device) == labels.to(device)).sum().data.cpu().numpy() / float(len(pred_))
#             test_loss_sum += test_loss.detach().cpu().numpy()
#             test_accuracy_n.append(test_acc)
#         examples_b.append(images.shape[0])
        
#         print('Train eval Loss: {:.4f}, Train eval Acc: {:.4f}'.format(np.mean(train_test_loss_sum/len(train_loader)/(batch_size)),
#                                                                                np.mean(train_test_accuracy_n)))

#         print('Test Loss: {:.4f}, Test Acc: {:.4f}'.format(np.mean(test_loss_sum/len(test_loader)/batch_size),
#                                                                                np.mean(test_accuracy_n)))
#     return np.mean(test_accuracy_n), np.mean(train_test_loss_sum/len(train_loader)/batch_size), np.mean(test_loss_sum/len(test_loader)/batch_size)

def precision_transfer(x, precision_bit=8, threshold=1.0):
    if len(x.shape) == 1:
        o = x.shape
    elif len(x.shape) == 2:
        i, o = x.shape
    else:
        i, o, row, col = x.shape
    x_list = x.reshape((-1,))
    
    A = 8
    max_w = threshold/(2**A/2**(precision_bit-1))
    min_w = -max_w
    step = np.diff(np.linspace(min_w, max_w, 2**precision_bit)[0:2])
    max_w = max_w - step[0]
    
#     n = 2**precision_bit - 1
#     qa = (min_w*2.0)/n
    q_list = np.round(np.linspace(min_w, max_w, 2**precision_bit), precision_bit - 2)
#     q_list[int(2**precision_bit)] = 0
    
    func = lambda x:q_list[np.abs(x - q_list).argmin()]
    q_list = np.array(list(map(func, x_list)))
    
    if len(x.shape) == 1:
        return q_list
    elif len(x.shape) == 2:
        return q_list.reshape((i, o))
    else:
        return q_list.reshape((i, o, row, col))
    
def low_precision(state_dict, precision=8, threshold=1.0):
    for k in state_dict.keys():
        if not isinstance(k, nn.MaxPool2d) and not isinstance(k, nn.AvgPool2d):
            w = precision_transfer(state_dict[k].data.cpu().numpy(), precision_bit=precision, threshold=threshold)
            w[state_dict[k].data.cpu().numpy() == 0] = 0
            state_dict[k] = torch.tensor(w)
    return state_dict

In [24]:
# torch.cuda.empty_cache()

In [25]:
def create_scheduler(args, optimizer):
    num_epochs = args.epochs

    if getattr(args, 'lr_noise', None) is not None:
        lr_noise = getattr(args, 'lr_noise')
        if isinstance(lr_noise, (list, tuple)):
            noise_range = [n * num_epochs for n in lr_noise]
            if len(noise_range) == 1:
                noise_range = noise_range[0]
        else:
            noise_range = lr_noise * num_epochs
    else:
        noise_range = None
    noise_args = dict(
        noise_range_t=noise_range,
        noise_pct=getattr(args, 'lr_noise_pct', 0.67),
        noise_std=getattr(args, 'lr_noise_std', 1.),
        noise_seed=getattr(args, 'seed', 42),
    )
    cycle_args = dict(
        cycle_mul=getattr(args, 'lr_cycle_mul', 1.),
        cycle_decay=getattr(args, 'lr_cycle_decay', 0.1),
        cycle_limit=getattr(args, 'lr_cycle_limit', 1),
    )

    lr_scheduler = None
    if args.sched == 'cosine':
        lr_scheduler = CosineLRScheduler(
            optimizer,
            t_initial=num_epochs,
            lr_min=args.min_lr,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            k_decay=getattr(args, 'lr_k_decay', 1.0),
            **cycle_args,
            **noise_args,
        )
        num_epochs = lr_scheduler.get_cycle_length() + args.cooldown_epochs
    elif args.sched == 'tanh':
        lr_scheduler = TanhLRScheduler(
            optimizer,
            t_initial=num_epochs,
            lr_min=args.min_lr,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            t_in_epochs=True,
            **cycle_args,
            **noise_args,
        )
        num_epochs = lr_scheduler.get_cycle_length() + args.cooldown_epochs
    elif args.sched == 'step':
        lr_scheduler = StepLRScheduler(
            optimizer,
            decay_t=args.decay_epochs,
            decay_rate=args.decay_rate,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            **noise_args,
        )
    elif args.sched == 'multistep':
        lr_scheduler = MultiStepLRScheduler(
            optimizer,
            decay_t=args.decay_milestones,
            decay_rate=args.decay_rate,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            **noise_args,
        )
    elif args.sched == 'plateau':
        mode = 'min' if 'loss' in getattr(args, 'eval_metric', '') else 'max'
        lr_scheduler = PlateauLRScheduler(
            optimizer,
            decay_rate=args.decay_rate,
            patience_t=args.patience_epochs,
            lr_min=args.min_lr,
            mode=mode,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            cooldown_t=0,
            **noise_args,
        )
    elif args.sched == 'poly':
        lr_scheduler = PolyLRScheduler(
            optimizer,
            power=args.decay_rate,  # overloading 'decay_rate' as polynomial power
            t_initial=num_epochs,
            lr_min=args.min_lr,
            warmup_lr_init=args.warmup_lr,
            warmup_t=args.warmup_epochs,
            k_decay=getattr(args, 'lr_k_decay', 1.0),
            **cycle_args,
            **noise_args,
        )
        num_epochs = lr_scheduler.get_cycle_length() + args.cooldown_epochs

    return lr_scheduler, num_epochs

In [26]:
device = torch.device("cuda:0")
model.to(device)
if dataset == 'genomic':
    if data_index == 3 or data_index == 4 or data_index == 2 or data_index == 7:
        learning_rate = 2e-3
    elif data_index == 6:
        learning_rate = 2e-3
    elif data_index == 5:
        learning_rate = 5e-3
    else:
        learning_rate = 1e-3
elif dataset == 'MNIST':
    learning_rate = 3e-3
elif dataset == "NMNIST":
    learning_rate = 3e-3
elif dataset == 'DVS128':
    learning_rate = 5e-3
elif dataset == 'cifar10':
    learning_rate = 1e-4
else:
    learning_rate = 1e-3
precision_epochs = 150
n_bits = 8
warmup_epochs = 10

In [27]:
seed = 0

In [28]:
path = folder +"dataset_" + str(data_index+1) + "_timestep_"+str(time_steps)+"_head_"+str(num_heads)+"mlp_ratio"+str(mlp_ratio)+f"spsmodule_{spsmodule}"+"_encoder_"+str(layers)+"_embed_dims_"+str(embed_dims)+"_nbits_"+str(n_bits)+"_ep_"+str(num_epoch)+"_lr(e-3)_"+str(learning_rate * 1000)+"_seed_"+str(seed)+".pth"

In [29]:
folder

'./result/DVS128/'

In [30]:
# path = folder +f"encoder_type_{encodermodule}.pth"

In [31]:
cooldown = 10
if dataset == "DVS128" :
    class Args:
        def __init__(self):
            self.epochs = num_epoch - cooldown
            self.sched = 'step'  
            self.min_lr = 1e-5
            self.warmup_lr = 3e-4
            self.warmup_epochs = 20
            self.decay_rate = 0.9
            self.cooldown_epochs = cooldown
            self.lr_noise = [0.6, 0.9]
            self.lr_noise_pct = 0.67
            self.lr_noise_std = 1.0
            self.seed = 42
            self.decay_epochs = 20
            self.patience_epochs = 5
elif dataset == "cifar10":
    class Args:
        def __init__(self):
            self.epochs = num_epoch - cooldown
            self.sched = 'step'  
            self.min_lr = 1e-5
            self.warmup_lr = 3e-5
            self.warmup_epochs = 5
            self.decay_rate = 0.9
            self.cooldown_epochs = cooldown
            self.lr_noise = [0.6, 0.9]
            self.lr_noise_pct = 0.67
            self.lr_noise_std = 1.0
            self.seed = 42
            self.decay_epochs = 20
            self.patience_epochs = 5 
else:
    class Args:
        def __init__(self):
            self.epochs = num_epoch - cooldown
            self.sched = 'cosine'  
            self.min_lr = 5e-5
            self.warmup_lr = 3e-4
            self.warmup_epochs = 20
            self.decay_rate = 0.99
            self.cooldown_epochs = cooldown
            self.lr_noise = [0.6, 0.9]
            self.lr_noise_pct = 0.67
            self.lr_noise_std = 1.0
            self.seed = 42
            self.decay_epochs = 20
            self.patience_epochs = 5

optimizer = torch.optim.Adam([{"params":base_params}, ], lr=learning_rate, weight_decay=1e-5)

args = Args()

In [32]:
optimizer = torch.optim.Adam([{"params":base_params}, ], lr=learning_rate, weight_decay=0)
# optimizer = Lamb(model.parameters(), lr=learning_rate, weight_decay=1e-4)
if dataset == "genomic":
#     scheduler = StepLR(optimizer, step_size=50, gamma=.6)
    scheduler, num_epochs = create_scheduler(args, optimizer)

#     scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.8, patience=5, verbose=True)
elif dataset == "MNIST" or dataset == "NMNIST":
#     scheduler = StepLR(optimizer, step_size=15, gamma=.8)
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.6, patience=3, verbose=True)
    scheduler, num_epochs = create_scheduler(args, optimizer)
elif dataset == "DVS128" or dataset == "cifar10":
#     scheduler = StepLR(optimizer, step_size=5, gamma=.9)
#     scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, verbose=True)
    scheduler, num_epochs = create_scheduler(args, optimizer)
else:
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=5, verbose=True)
#     scheduler = StepLR(optimizer, step_size=10, gamma=.9)
    scheduler, num_epochs = create_scheduler(args, optimizer)

#     lambda1 = lambda epoch: 5 * epoch / 10 if epoch < 10 else 0.95 ** (epoch - 10)
#     scheduler = LambdaLR(optimizer, lr_lambda=lambda1)
#     scheduler = create_scheduler(optimizer, 'cosine', num_epoch=num_epoch, decay_epochs=90, 
#                               cooldown_epochs=cooldown_epochs, decay_rate=0.1, min_lr=1e-10, warmup_lr=1e-6, 
#                               warmup_epochs=10, plateau_mode='max')
# scheduler = lr_lambda(optimizer, T_max=num_epoch/2)

In [33]:
#wandb optimization sweep hyperparameters
import wandb

In [34]:
wandb.login(key='36511cd28a79d90c5bc87bc5745b7bff232c3eeb')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: federico-corradi (necs_lab). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/federico/.netrc


True

In [38]:
sweep_config = {
    'method': 'bayes',  # Bayesian optimization to efficiently explore the hyperparameter space
    'metric': {
        'name': 'test_accuracy',  # Primary metric to optimize
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-4,
            'max': 1e-3  # Learning rate range
        },
        'batch_size': {
            'values': [12,16]  # Possible batch sizes
        },
        'time_steps': {
            'values': [16]  # Add more values if needed
        },
        'embed_dims': {
            'values': [256]  # Common options for embedding dimensions
        },
        'patch_size': {
            'values': [8]  # Example values for patch sizes
        },
        'input_extend': {
            'values': [1,4,8]  # Binary choice
        },
        'num_epoch': {
            'values': [300]  # Experiment with different training durations
        },
        'thresholds': {
            'values': [
                [128/128, 128/128, 128/128],  # Current configuration
                [64/128, 64/128, 64/128]    # Example alternative
                #[192/128, 192/128, 192/128]  # Another example
            ]
        },
        'pooling_stat': {
            'values': ["1111"]  # Example configurations
        },
        'manual_seed': {
            'min': 1,
            'max': 250,  # Define the range of seeds to explore
            'distribution': 'int_uniform'  # Ensure seeds are integers
        }
    }
}


In [39]:
import wandb
from tqdm import tqdm

# Initialize wandb run
wandb.init(project="SpikeVisionSeedDBv1", config={
    "learning_rate": 1e-3,
    "batch_size": 128,
    "num_epochs": 50,
    "embed_dim": 256,
    "alpha": 0.9,
    "v_threshold": 0.1,
    "scheduler_type": "cosine"
})

In [40]:
def custom_collate_fn(batch):
    max_timesteps = time_steps
    padded_batch = []

    for item in batch:
        data, label = item
        if isinstance(data, np.ndarray):
            data = torch.from_numpy(data).float()

        current_timesteps = data.size(0)

        if current_timesteps < max_timesteps:
            padding_size = (0, 0, 0, 0, 0, 0, 0, max_timesteps - current_timesteps)
            padded_data = torch.nn.functional.pad(data, pad=padding_size, mode='constant', value=0)
        else:
            padded_data = data[:max_timesteps]

        padded_batch.append((padded_data, label))
    return torch.utils.data.dataloader.default_collate(padded_batch)

In [ ]:
def train():
    # Initialize wandb
    with wandb.init() as run:
        config = wandb.config

        # Set default hyperparameters if not provided in sweep config
        if not hasattr(config, 'num_epochs'):
            config.num_epochs = 250
        if not hasattr(config, 'batch_size'):
            config.batch_size = 20
        if not hasattr(config, 'learning_rate'):
            config.learning_rate = 1e-3
        if not hasattr(config, 'embed_dims'):
            config.embed_dims = 256
        if not hasattr(config, 'time_steps'):
            config.time_steps = 16
        if not hasattr(config, 'patch_size'):
            config.patch_size = 8
        if not hasattr(config, 'input_extend'):
            config.input_extend = 1
        if not hasattr(config, 'thresholds'):
            config.thresholds = [128/128, 128/128, 128/128]
        if not hasattr(config, 'pooling_stat'):
            config.pooling_stat = "1111"
        if not hasattr(config, 'mlp_ratio'):
            config.mlp_ratio = 2
        if not hasattr(config, 'num_heads'):
            config.num_heads = 1
        if not hasattr(config, 'layers'):
            config.layers = 2
        if not hasattr(config, 'train_threshold'):
            config.train_threshold = False
        if not hasattr(config, 'image_size'):
            config.image_size = 128
        if not hasattr(config, 'image_size_w'):
            config.image_size_w = 128
        if not hasattr(config, 'in_channels'):
            config.in_channels = 2
        if not hasattr(config, 'num_classes'):
            config.num_classes = 11
        if not hasattr(config, 'manual_seed'):
            config.manual_seed = 49

        # Data Preparation
        def binarize(tensor):
            return (tensor > 0.5).float()

        transform = transforms.Compose([
            transforms.ToTensor(),
            binarize
        ])

        # Combine and split dataset
        combined_dataset = ConcatDataset([
            DVS128Gesture(
                "data/DVS128/",
                train=True,
                data_type="frame",
                custom_integrate_function=integrate_fixed_15000_events
            ),
            DVS128Gesture(
                "data/DVS128/",
                train=False,
                data_type="frame",
                custom_integrate_function=integrate_fixed_15000_events
            )
        ])
        
        train_size = int(0.8 * len(combined_dataset))
        test_size = len(combined_dataset) - train_size
        
        # Use a deterministic generator with a manual seed
        generator = torch.Generator()
        seed = np.random.randint(low=0, high=100)
        print(f"Random Seed: {seed}")
        generator.manual_seed(config.manual_seed)
        
        dataset_train, dataset_test = random_split(combined_dataset, [train_size, test_size], generator=generator)
        
        # Create DataLoaders
        train_loader = DataLoader(
            dataset_train,
            batch_size=config.batch_size,
            shuffle=True,
            collate_fn=custom_collate_fn,
            num_workers=num_workers,
            pin_memory=False,
        )
        
        test_loader = DataLoader(
            dataset_test,
            batch_size=config.batch_size,
            shuffle=False,
            collate_fn=custom_collate_fn,
            num_workers=num_workers,
            pin_memory=False,
        )
        
        print(f"Train Loader Length: {len(train_loader)}")
        print(f"Test Loader Length: {len(test_loader)}")
    

        # Model instantiation
        model = SpikeVisionModel(
            T=config.time_steps,
            img_size_h=config.image_size,
            img_size_w=config.image_size_w,
            in_channels=config.in_channels,
            num_classes=config.num_classes,
            patch_size=config.patch_size,
            embed_dims=config.embed_dims,
            num_heads=config.num_heads,
            pooling_stat=config.pooling_stat,
            mlp_ratio=config.mlp_ratio,
            threshold_head=config.thresholds[0],
            threshold_sps=config.thresholds[1],
            threshold_enc=config.thresholds[2],
            depths=config.layers,
            train_threshold=config.train_threshold       
        )

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

        # Training Loop
        num_epochs = config.num_epochs
        best_accuracy = 0.0

        for epoch in range(num_epochs):
            # TRAIN
            model.train()
            train_loss_sum = 0.0
            train_correct = 0
            total_train_samples = 0

            for images, labels in train_loader:
                images = images.to(device)  # Move images to device
                labels = labels.to(device)  # Move labels to device

                # Reshape images for spiking classifier
                images = images.squeeze(1)  # Remove channel dimension
                #images = images.permute(1, 0, 2, 3, 4)  # Convert [B, T, C, H, W] to [T, B, C, H, W]

                # Forward pass
                outputs, _, _ = model(images)  # Outputs: [B, num_classes]
                #print(f"Logits shape: {outputs.shape}, Labels shape: {labels.shape}")
                #print(f"Labels: {labels}")
                loss = criterion(outputs, labels)
            

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Metrics
                train_loss_sum += loss.item() * labels.size(0)
                train_correct += (outputs.argmax(1) == labels).sum().item()
                total_train_samples += labels.size(0)

            train_loss = train_loss_sum / total_train_samples
            train_accuracy = 100. * train_correct / total_train_samples

            # Evaluation
            model.eval()
            test_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs, _, _ = model(images)  # Extract only logits
                    test_loss += criterion(outputs, labels).item()
                    _, predicted = outputs.max(1)  # Get predicted classes
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

            test_loss = test_loss / total
            test_accuracy = 100. * correct / total

            # Log metrics and hyperparameters to wandb
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_accuracy": train_accuracy,
                "test_loss": test_loss,
                "test_accuracy": test_accuracy,
                "learning_rate": config.learning_rate,
                "batch_size": config.batch_size,
                "time_steps": config.time_steps,
                "embed_dims": config.embed_dims,
                "patch_size": config.patch_size,
                "input_extend": config.input_extend,
                "num_epoch": config.num_epochs,
                "thresholds": config.thresholds,
                "pooling_stat": config.pooling_stat,
                "mlp_ratio": config.mlp_ratio,
                "num_heads": config.num_heads,
                "layers": config.layers,
                "train_threshold": config.train_threshold,
                "manual_seed": config.manual_seed,
            })

            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
                  f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

            # Stop if accuracy at epoch 20 is below 84%
            if epoch + 1 == 20 and test_accuracy < 73.0:
                print(f"Test accuracy below 73% at epoch 20. Stopping current training run.")
                return  # Ends current `train` function, letting wandb agent start a new run.

            # Save best model
            if (test_accuracy > best_accuracy) and (test_accuracy > 88):
                best_accuracy = test_accuracy
                # Include configuration parameters in the filename
                model_save_path = (
                    f"savedmodels/tsdt_model_epoch{epoch+1}_acc{test_accuracy:.2f}_"
                    f"lr{config.learning_rate}_bs{config.batch_size}_tsteps{config.time_steps}_"
                    f"embed{config.embed_dims}_patch{config.patch_size}_ext{config.input_extend}_"
                    f"seeddb{config.manual_seed}_seeddb_"
                    f"layers{config.layers}_heads{config.num_heads}_mlp{config.mlp_ratio}.pth"
                )
                torch.save(model.state_dict(), model_save_path)
                wandb.save(model_save_path)
                print(f'New best model saved with test accuracy: {best_accuracy:.2f}%')

        print(f'Training complete. Best test accuracy: {best_accuracy:.2f}%')

# Start the sweep
sweep_id = wandb.sweep(sweep_config, project="SpikeVisionSeedDBv1")
wandb.agent(sweep_id, function=train, count=20)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: amzq1zqt
Sweep URL: https://wandb.ai/necs_lab/SpikeVisionSeedDBv1/sweeps/amzq1zqt


wandb: Agent Starting Run: 3hr21ex0 with config:
wandb: 	batch_size: 16
wandb: 	embed_dims: 256
wandb: 	input_extend: 8
wandb: 	learning_rate: 0.0009544711456481556
wandb: 	manual_seed: 196
wandb: 	num_epoch: 300
wandb: 	patch_size: 8
wandb: 	pooling_stat: 1111
wandb: 	thresholds: [0.5, 0.5, 0.5]
wandb: 	time_steps: 16


wandb: 🚀 View run splendid-shape-2 at: https://wandb.ai/necs_lab/SpikeVisionSeedDBv1/runs/48sparki
wandb: Find logs at: wandb/run-20241202_125929-48sparki/logs


The directory [data/DVS128/integrate_fixed_15000_events] already exists.
The directory [data/DVS128/integrate_fixed_15000_events] already exists.
Random Seed: 67
Train Loader Length: 74
Test Loader Length: 19
Epoch [1/250], Train Loss: 2.3979, Train Accuracy: 8.71%, Test Loss: 0.1456, Test Accuracy: 15.70%
Epoch [2/250], Train Loss: 1.9918, Train Accuracy: 33.65%, Test Loss: 0.1329, Test Accuracy: 55.63%
Epoch [3/250], Train Loss: 1.6200, Train Accuracy: 48.08%, Test Loss: 0.1703, Test Accuracy: 46.08%
Epoch [4/250], Train Loss: 1.4090, Train Accuracy: 55.42%, Test Loss: 0.1328, Test Accuracy: 58.02%
Epoch [5/250], Train Loss: 1.4216, Train Accuracy: 55.59%, Test Loss: 0.1117, Test Accuracy: 61.77%
Epoch [6/250], Train Loss: 1.2783, Train Accuracy: 56.02%, Test Loss: 0.1255, Test Accuracy: 57.68%
Epoch [7/250], Train Loss: 1.2226, Train Accuracy: 59.44%, Test Loss: 0.1179, Test Accuracy: 59.39%
Epoch [8/250], Train Loss: 1.1996, Train Accuracy: 59.44%, Test Loss: 0.1053, Test Accuracy:

batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
embed_dims,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇██████
input_extend,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layers,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
manual_seed,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mlp_ratio,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_heads,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
patch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3osw89lt with config:
wandb: 	batch_size: 16
wandb: 	embed_dims: 256
wandb: 	input_extend: 8
wandb: 	learning_rate: 0.000976521069180304
wandb: 	manual_seed: 197
wandb: 	num_epoch: 300
wandb: 	patch_size: 8
wandb: 	pooling_stat: 1111
wandb: 	thresholds: [1, 1, 1]
wandb: 	time_steps: 16


The directory [data/DVS128/integrate_fixed_15000_events] already exists.
The directory [data/DVS128/integrate_fixed_15000_events] already exists.
Random Seed: 30
Train Loader Length: 74
Test Loader Length: 19
Epoch [1/250], Train Loss: 2.3979, Train Accuracy: 8.11%, Test Loss: 0.1555, Test Accuracy: 8.87%
Epoch [2/250], Train Loss: 2.3979, Train Accuracy: 8.11%, Test Loss: 0.1485, Test Accuracy: 15.02%
Epoch [3/250], Train Loss: 2.0092, Train Accuracy: 31.51%, Test Loss: 0.1280, Test Accuracy: 48.12%


In [ ]:
#print(f"Using model: {model_type}, sps module: {spsmodule}, encoder module: {encodermodule}. Seed: {seed}")
# model.half()
# ipdb.set_trace()
#load=0
#if load:
#    precision_epochs = 10
#train(
#    model, 
#    loader_train, 
#    loader_test, 
#    optimizer, 
#    scheduler, 
#    num_epochs=num_epoch,
#    chan=in_channels,
#    precision_epochs=precision_epochs,
#    load=load,
#    path=path,
#)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_loss_list, label='Train Loss')
# plt.plot(train_eval_loss_list, label='Train eval Loss')
plt.plot(test_loss_list, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
path_low = folder +"dataset_" + str(data_index+1) + "_timestep_"+str(time_steps)+"_head_"+str(num_heads)+"_mlp_ratio_"+str(mlp_ratio)+"_encoder_"+str(layers)+"_embed_dims_"+str(embed_dims)+"_nbits_"+str(n_bits)+"_ep_"+str(num_epoch)+"_lr(e-3)_"+str(learning_rate * 1000)+ "_low_precision"+".pth"

base_params

In [ ]:
# state_dict = torch.load('DVS128best99_65_2layer_8.pth')
# state_dict = torch.load('DVS128best97_95_2layer.pth')
state_dict = torch.load("result/DVS128/DVS128best98_29_2layer.pth")
#state_dict = torch.load(path_low)
model.load_state_dict(state_dict)
model = model.to(device)
test_acc, train_eval_acc, test_loss = test(model, loader_train, loader_test)
test_acc

In [ ]:
seeds = []
test_accs = []
def custom_collate_fn(batch):
    max_timesteps = 8 
    padded_batch = []

    for item in batch:
        data, label = item
        if isinstance(data, np.ndarray):
            data = torch.from_numpy(data).float()

        current_timesteps = data.size(0)

        if current_timesteps < max_timesteps:
            padding_size = (0, 0, 0, 0, 0, 0, 0, max_timesteps - current_timesteps)
            padded_data = torch.nn.functional.pad(data, pad=padding_size, mode='constant', value=0)
        else:
            padded_data = data[:max_timesteps]

        padded_batch.append((padded_data, label))

    return torch.utils.data.dataloader.default_collate(padded_batch)
for seed in tqdm(range(100)):
    generator.manual_seed(seed)
    dataset_train, dataset_test = random_split(combined_dataset, [train_size, test_size], generator=generator)
    
    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    loader_test = DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    test_acc, train_eval_acc, test_loss = test(model, loader_train, loader_test)
    test_accs.append(test_acc)

for i in range(100):
    print(i, test_accs[i])

In [ ]:
for i in range(len(test_accs)):
    if test_accs[i] == 0.9829351535836177:
        print(i)

In [ ]:
import random
step_accs = []
generator.manual_seed(49)
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# dataset_train, dataset_test = random_split(combined_dataset, [train_size, test_size], generator=generator)
# loader_train = DataLoader(
#     dataset_train,
#     batch_size=batch_size,
#     shuffle=True,
#     collate_fn=custom_collate_fn,
#     num_workers=num_workers,
#     pin_memory=False,
# )
# loader_test = DataLoader(
#     dataset_test,
#     batch_size=batch_size,
#     shuffle=True,
#     collate_fn=custom_collate_fn,
#     num_workers=num_workers,
#     pin_memory=False,
# )
dataset_train, dataset_test = random_split(combined_dataset, [train_size, test_size], generator=generator)
for max_timesteps in tqdm(range(1, 32)):
    def custom_collate_fn(batch):
    
        padded_batch = []
    
        for item in batch:
            data, label = item
            if isinstance(data, np.ndarray):
                data = torch.from_numpy(data).float()
    
            current_timesteps = data.size(0)
    
            if current_timesteps < max_timesteps:
                padding_size = (0, 0, 0, 0, 0, 0, 0, max_timesteps - current_timesteps)
                padded_data = torch.nn.functional.pad(data, pad=padding_size, mode='constant', value=0)
            else:
                padded_data = data[:max_timesteps]
    
            padded_batch.append((padded_data, label))
    
        return torch.utils.data.dataloader.default_collate(padded_batch)
    
    
    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    loader_test = DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    test_acc, train_eval_acc, test_loss = test(model, loader_train, loader_test)
    step_accs.append(test_acc)
for i in range(1, 32):
    print(f"Time step amount: {i}, Test accuracy: {step_accs[i - 1]}")

In [ ]:
plt.figure(figsize=(25, 10))
plt.plot(range(1, 27), step_accs[0:26], 'r', marker='o')
plt.xlabel('Time steps', fontsize=25, fontweight='heavy')
plt.ylabel('Accuracy', fontsize=25, fontweight='heavy')
plt.xticks(fontsize=18, fontweight='heavy')
plt.yticks(fontsize=18, fontweight='heavy')

# for i, v in enumerate(step_accs):
#     # plt.vlines(i+1, 0, v, linestyles='dashed', colors='gray')
    
#     plt.text(i+1, v, f'{v:.4f}', ha='center', va='bottom', fontsize=20)
previous_y = None 
for i, v in enumerate(step_accs):
    if i < 26:
        height = v
        if previous_y is not None and abs(previous_y - height) < 0.01:
            height = v + 0.01
    
        plt.text(i + 1, height, f'{v:.3f}', ha='center', va='bottom', fontsize=15)
        previous_y = height
plt.ylim(0.7, 1.02)
plt.savefig('time_step_acc.png', dpi=500)
plt.show()

In [ ]:
arrays_order = []

for max_timesteps in tqdm(range(1, 3)):
    def custom_collate_fn(batch):
    
        padded_batch = []
    
        for item in batch:
            data, label = item
            if isinstance(data, np.ndarray):
                data = torch.from_numpy(data).float()
    
            current_timesteps = data.size(0)
    
            if current_timesteps < max_timesteps:
                padding_size = (0, 0, 0, 0, 0, 0, 0, max_timesteps - current_timesteps)
                padded_data = torch.nn.functional.pad(data, pad=padding_size, mode='constant', value=0)
            else:
                padded_data = data[:max_timesteps]
    
            padded_batch.append((padded_data, label))
    
        return torch.utils.data.dataloader.default_collate(padded_batch)
    
    
    loader_train = DataLoader(
        dataset_train,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    loader_test = DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=custom_collate_fn,
        num_workers=num_workers,
        pin_memory=False,
    )
    example = enumerate(loader_test)
    _, (tensors, labels) = next(example)
    arrays_order.append(tensors[0])

In [ ]:
(arrays_order[0][0] == arrays_order[1][0]).all()

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():  
    for images, labels in loader_test:
        outputs, _, _ = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())  
        y_pred.extend(predicted.cpu().numpy())  

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


conf_mat = confusion_matrix(y_true, y_pred)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]


plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_normalized, annot=True, fmt='.2f', cmap='Reds', xticklabels=list(range(11)), yticklabels=list(range(11)))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for DVS128')
plt.savefig('confusion_matrix_DVS128.png')

plt.show()

In [ ]:
# torch.save(model.state_dict(), 'DVS128best95_90_2layer_seed'+str(seed)+'.pth')
torch.save(model.state_dict(), f'DVS128_smaller_model.pth')

In [ ]:
%debug

In [ ]:
c_path = folder + f"c_implementation/"
c_path

In [ ]:
threshold = 2**n_bits
print(threshold)
ss_dict = model.state_dict()
for name, param in model.state_dict().items():
    param = (param*threshold).data.cpu().numpy()
    param = np.round(param)
    ss_dict[i] = torch.tensor(param)
    print(f"Layer: {name} | Shape: {param.shape}")
    print(param, "\n")  
    weights_1d = param.flatten()
    

    np.savetxt(c_path + f"{name}_weights.txt", weights_1d, fmt="%d")
    
    # 打印确认
    print(f"Layer: {name} | Saved shape: {weights_1d.shape}")
# ss_dict = model.state_dict()
# for i in ss_dict.keys():
#     key = (ss_dict[i]*threshold).data.cpu().numpy()
#     key = np.round(key)
#     ss_dict[i] = torch.tensor(key)#, dtype = torch.int)
    
print(ss_dict)

In [ ]:
    weights_1d = param.view(-1).cpu().numpy()  # 转换为一维数组
    
    # 保存到文件，每一层的权重保存为独立的文件
    np.savetxt(f"{name}_weights.txt", weights_1d, fmt="%.6f")
    
    # 打印确认
    print(f"Layer: {name} | Saved shape: {weights_1d.shape}")

In [ ]:
with open(c_path + 'input_data.txt', 'r') as file:
    data = file.readlines()

# 转换为 NumPy 数组
data_array = np.array([list(map(float, line.split())) for line in data])
reshaped_arr = data_array.reshape(20, 8, 2, 128, 128)
transposed_arr = reshaped_arr.transpose(0, 1, 3, 4, 2)
final_arr = transposed_arr.reshape(20, 8, 128, 256)

In [ ]:
reshaped_final_arr = final_arr.reshape(160*128, 256)
np.savetxt(c_path + 'buffer_output.txt', reshaped_final_arr, fmt='%d', delimiter=' ')